In [1]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [1784]:
category = "literal"
idiom = "make hit"
word1 = "make"
word2 = "hit"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [1785]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [1786]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

In [1787]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [1788]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!


In [1789]:
type = "Zero-shot"
step = 12

In [1790]:
idiom.split(" ")
sentences = ""
sonx = idiom.split(" ")[-1]
firstx = idiom.split(" ")[0]
for l in idiom.split(" "):
    if sonx == l:
        sentences = sentences +" and "+'"'+ l.lower()+ '"'
    elif firstx == l:
        sentences = sentences +" "+'"'+ l.lower() + '"'
    else:
        sentences = sentences +", "+'"'+ l.lower() + '"'

In [1791]:
prompt = f"""
Write 5 example sentences that use the words "{word1}" and "{word2}" to express a physical action. You may conjugate the specified words in any form you like without changing their roots. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Only return the sentences in your response.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Write 5 example sentences that use the words "make" and "hit" to express a physical action. You may conjugate the specified words in any form you like without changing their roots. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Only return the sentences in your response.





In [1792]:
responses

['1. During the baseball game, he made a fast pitch, then quickly hit the ball, sending it soaring into the stands.\n2. In the workshop, they hit the metal with hammers to shape it after making precise measurements.\n3. She made a small dent in the surface and then hit it with a punch to start drilling.\n4. To ensure the structure was sturdy, the engineer made calculations before hitting the foundation piles into the ground.\n5. As part of the demonstration, the chef made a dough ball and then hit it flat to show how to prepare a pizza base.',
 '1. After making a tight fist, the boxer hit the punching bag with remarkable force.\n2. During the renovation, they hit the old walls with sledgehammers after making sure the area was clear of wiring.\n3. In the smithy, the artisan made careful adjustments to the metal before hitting it with a heavy mallet.\n4. The drummer made a quick adjustment to the snare before hitting it to test the sound during rehearsal.\n5. The kids made snowballs and 

In [1793]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [1794]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
5,"After making a tight fist, the boxer hit the p...",literal,make hit,Zero-shot
6,"During the renovation, they hit the old walls ...",literal,make hit,Zero-shot
7,"In the smithy, the artisan made careful adjust...",literal,make hit,Zero-shot
8,The drummer made a quick adjustment to the sna...,literal,make hit,Zero-shot
9,The kids made snowballs and hit the side of th...,literal,make hit,Zero-shot


In [1795]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
5,"After making a tight fist, the boxer hit the p...",literal,make hit,Zero-shot
6,"During the renovation, they hit the old walls ...",literal,make hit,Zero-shot
7,"In the smithy, the artisan made careful adjust...",literal,make hit,Zero-shot
8,The drummer made a quick adjustment to the sna...,literal,make hit,Zero-shot
9,The kids made snowballs and hit the side of th...,literal,make hit,Zero-shot


In [1796]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
5,"After making a tight fist, the boxer hit the p...",literal,make hit,Zero-shot
6,"During the renovation, they hit the old walls ...",literal,make hit,Zero-shot
7,"In the smithy, the artisan made careful adjust...",literal,make hit,Zero-shot
8,The drummer made a quick adjustment to the sna...,literal,make hit,Zero-shot
9,The kids made snowballs and hit the side of th...,literal,make hit,Zero-shot


In [1797]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [1798]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!


In [1799]:
type = "Enhanced-prompting"
step = 4

In [1800]:
prompt = f"""
Act like human and write 5 example sentences that use the words "{word1}" and "{word2}" to express a physical action. You should use the words in their literal sense. You may conjugate the specified words in any form you like without changing their roots. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Please avoid repeating your sentences. Only return the sentences in your response.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Act like human and write 5 example sentences that use the words "make" and "hit" to express a physical action. You should use the words in their literal sense. You may conjugate the specified words in any form you like without changing their roots. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Please avoid repeating your sentences. Only return the sentences in your response.





In [1801]:
responses

["1. The chef made a thin crust pizza before hitting the oven's hot surface accidentally with his hand.\n2. To test its durability, they made a new model of the smartphone and then hit it with a hammer.\n3. She made a snowball and hit the tree trunk with it from twenty feet away.\n4. In the workshop, they made a metal sculpture and hit it repeatedly to get the desired shape.\n5. During the karate practice, he made precise movements before hitting the training pad squarely.",
 '1. In the pottery class, she made a clay pot before hitting it accidentally against the table, causing it to break.\n2. After making a special mixture for the punching bag, he hit it hard to test its resistance.\n3. The children made a large fortress out of cardboard and hit each part to ensure it was sturdy enough for their game.\n4. The carpenter made a wooden doll and hit the joints to ensure all parts were properly attached.\n5. During the baseball game, he made a swift swing and hit the ball out of the park.

In [1802]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,The chef made a thin crust pizza before hittin...,literal,make hit,Enhanced-prompting
1,"To test its durability, they made a new model ...",literal,make hit,Enhanced-prompting
2,She made a snowball and hit the tree trunk wit...,literal,make hit,Enhanced-prompting
3,"In the workshop, they made a metal sculpture a...",literal,make hit,Enhanced-prompting
4,"During the karate practice, he made precise mo...",literal,make hit,Enhanced-prompting
5,"In the pottery class, she made a clay pot befo...",literal,make hit,Enhanced-prompting
6,After making a special mixture for the punchin...,literal,make hit,Enhanced-prompting
7,The children made a large fortress out of card...,literal,make hit,Enhanced-prompting
8,The carpenter made a wooden doll and hit the j...,literal,make hit,Enhanced-prompting
9,"During the baseball game, he made a swift swin...",literal,make hit,Enhanced-prompting


In [1803]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,The chef made a thin crust pizza before hittin...,literal,make hit,Enhanced-prompting
1,"To test its durability, they made a new model ...",literal,make hit,Enhanced-prompting
2,She made a snowball and hit the tree trunk wit...,literal,make hit,Enhanced-prompting
3,"In the workshop, they made a metal sculpture a...",literal,make hit,Enhanced-prompting
4,"During the karate practice, he made precise mo...",literal,make hit,Enhanced-prompting
5,"In the pottery class, she made a clay pot befo...",literal,make hit,Enhanced-prompting
6,After making a special mixture for the punchin...,literal,make hit,Enhanced-prompting
7,The children made a large fortress out of card...,literal,make hit,Enhanced-prompting
8,The carpenter made a wooden doll and hit the j...,literal,make hit,Enhanced-prompting
9,"During the baseball game, he made a swift swin...",literal,make hit,Enhanced-prompting


In [1804]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
...,...,...,...,...
75,The batter made a powerful swing and hit the b...,literal,make hit,Enhanced-prompting
76,After making a slight adjustment to the micros...,literal,make hit,Enhanced-prompting
77,"During the game, he made a quick dodge and hit...",literal,make hit,Enhanced-prompting
78,She made a jump for the rope swing and hit the...,literal,make hit,Enhanced-prompting


In [1805]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [1806]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences. You should not construct sentences using personal names! You must use the words specified in the request in a real, concrete, or physical sense. You should not use the specified words in a metaphorical sense!


In [1807]:
type = "Enhanced-prompting"
step = 4

In [1808]:
prompt = f"""
Create 5 sentences with different structures, using the words "{word1}" and "{word2}" to express a physical action. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Please avoid repeating your sentences. Only return the sentences in your response.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Create 5 sentences with different structures, using the words "make" and "hit" to express a physical action. You can place the specified words anywhere in the sentence, but you must construct sentences that include both words. Please avoid repeating your sentences. Only return the sentences in your response.





In [1809]:
responses

['1. During the crafting workshop, participants learn to make a pottery vase before they hit the clay with paddles to shape it.\n\n2. The young boxer hit the punching bag forcefully after she decided to make boxing her primary sport.\n\n3. At the fair, while trying to make a new record, he hit the hammer down onto the strength tester.\n\n4. She decided to make an impression on her coaches, so she hit every ball pitched at her during the tryouts.\n\n5. To make the best use of the old desk, he hit the stuck drawer with force to loosen it.',
 '1. Could you make sure to hit the drum when I give the signal?\n2. When attempting to make the gate more secure, she hit the loose panel with a mallet.\n3. Ill-prepared and frantic, he hit the water hard as he tried to make the dive team.\n4. In an effort to make repairs, they hit the bent metal with hammers until it straightened.\n5. She hit the nail with precise force to make sure it went in straight.',
 '1. In the workshop, as they aimed to make 

In [1810]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"During the crafting workshop, participants lea...",literal,make hit,Enhanced-prompting
1,,literal,make hit,Enhanced-prompting
2,The young boxer hit the punching bag forcefull...,literal,make hit,Enhanced-prompting
3,,literal,make hit,Enhanced-prompting
4,"At the fair, while trying to make a new record...",literal,make hit,Enhanced-prompting
5,,literal,make hit,Enhanced-prompting
6,She decided to make an impression on her coach...,literal,make hit,Enhanced-prompting
7,,literal,make hit,Enhanced-prompting
8,"To make the best use of the old desk, he hit t...",literal,make hit,Enhanced-prompting
9,Could you make sure to hit the drum when I giv...,literal,make hit,Enhanced-prompting


In [1811]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,"During the crafting workshop, participants lea...",literal,make hit,Enhanced-prompting
1,The young boxer hit the punching bag forcefull...,literal,make hit,Enhanced-prompting
2,"At the fair, while trying to make a new record...",literal,make hit,Enhanced-prompting
3,She decided to make an impression on her coach...,literal,make hit,Enhanced-prompting
4,"To make the best use of the old desk, he hit t...",literal,make hit,Enhanced-prompting
5,Could you make sure to hit the drum when I giv...,literal,make hit,Enhanced-prompting
6,"When attempting to make the gate more secure, ...",literal,make hit,Enhanced-prompting
7,"Ill-prepared and frantic, he hit the water har...",literal,make hit,Enhanced-prompting
8,"In an effort to make repairs, they hit the ben...",literal,make hit,Enhanced-prompting
9,She hit the nail with precise force to make su...,literal,make hit,Enhanced-prompting


In [1812]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
...,...,...,...,...
95,"To correct the alignment, they had to hit the ...",literal,make hit,Enhanced-prompting
96,He hit the start button hard to make the old e...,literal,make hit,Enhanced-prompting
97,"Every time they make a batch of dough, she hit...",literal,make hit,Enhanced-prompting
98,She was eager to make a good impression at the...,literal,make hit,Enhanced-prompting


In [1813]:
df_structured

,submission,category,idiom,type
0,"During the baseball game, he made a fast pitch...",literal,make hit,Zero-shot
1,"In the workshop, they hit the metal with hamme...",literal,make hit,Zero-shot
2,She made a small dent in the surface and then ...,literal,make hit,Zero-shot
3,"To ensure the structure was sturdy, the engine...",literal,make hit,Zero-shot
4,"As part of the demonstration, the chef made a ...",literal,make hit,Zero-shot
...,...,...,...,...
95,"To correct the alignment, they had to hit the ...",literal,make hit,Enhanced-prompting
96,He hit the start button hard to make the old e...,literal,make hit,Enhanced-prompting
97,"Every time they make a batch of dough, she hit...",literal,make hit,Enhanced-prompting
98,She was eager to make a good impression at the...,literal,make hit,Enhanced-prompting


In [1814]:
# Define the path for the CSV file
file_path = 'gpt4_en_final_gerçek.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)